# <center> Лабораторная 2 </center>
## <center> Кластеризация</center>

### Набор данных с датчиков городских очистных сооружений сточных вод
####Цель - кластеризовать рабочее состояние установки для прогнозирования неисправностей с помощью алгоритмов k-means++ и dbscan water-treatment.data - выборка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.model_selection import train_test_split
from sklearn.metrics import v_measure_score, adjusted_rand_score
from sklearn.preprocessing import StandardScaler
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

В water-treatment.data дописаны две колонки с информацией из water-treatment.names (раздел 9). Колонку с датами удаляем.

In [5]:
init_data = pd.read_csv("/content/drive/MyDrive/water-treatment.data", delimiter=",", header = None)
data = init_data.drop([0], axis=1)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 527 entries, 0 to 526
Data columns (total 40 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   1       527 non-null    object 
 1   2       527 non-null    object 
 2   3       527 non-null    float64
 3   4       527 non-null    object 
 4   5       527 non-null    object 
 5   6       527 non-null    object 
 6   7       527 non-null    object 
 7   8       527 non-null    object 
 8   9       527 non-null    int64  
 9   10      527 non-null    float64
 10  11      527 non-null    object 
 11  12      527 non-null    int64  
 12  13      527 non-null    object 
 13  14      527 non-null    object 
 14  15      527 non-null    int64  
 15  16      527 non-null    float64
 16  17      527 non-null    object 
 17  18      527 non-null    object 
 18  19      527 non-null    object 
 19  20      527 non-null    object 
 20  21      527 non-null    object 
 21  22      527 non-null    int64  
 22  23

Делаем преобразование object в число для работы с числовыми значениями


In [7]:
col = [c for c in data.columns if data[c].dtype.name == 'object']
for c in col:
  data[c] = pd.to_numeric(data[c].astype(str), errors='coerce')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 527 entries, 0 to 526
Data columns (total 40 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   1       509 non-null    float64
 1   2       524 non-null    float64
 2   3       527 non-null    float64
 3   4       504 non-null    float64
 4   5       521 non-null    float64
 5   6       526 non-null    float64
 6   7       516 non-null    float64
 7   8       502 non-null    float64
 8   9       527 non-null    int64  
 9   10      527 non-null    float64
 10  11      487 non-null    float64
 11  12      527 non-null    int64  
 12  13      516 non-null    float64
 13  14      503 non-null    float64
 14  15      527 non-null    int64  
 15  16      527 non-null    float64
 16  17      499 non-null    float64
 17  18      518 non-null    float64
 18  19      525 non-null    float64
 19  20      514 non-null    float64
 20  21      502 non-null    float64
 21  22      527 non-null    int64  
 22  23

Даты, у которых нет информации в names, удаляем

In [8]:
init_data.drop([505, 506, 507, 508], inplace=True)
data.drop([505, 506, 507, 508], inplace=True)

In [9]:
data

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
0,44101.0,1.50,7.8,NaN,407.0,166.0,66.3,4.5,2110,7.9,NaN,228,70.2,5.5,2120,7.9,NaN,280.0,94.0,72.3,0.3,2010,7.3,NaN,84.0,21.0,81.0,0.02,2000.0,NaN,58.8,95.5,NaN,70.0,NaN,79.4,87.3,99.6,1.0,NaN
1,39024.0,3.00,7.7,NaN,443.0,214.0,69.2,6.5,2660,7.7,NaN,244,75.4,7.7,2570,7.6,NaN,474.0,96.0,79.2,0.4,2700,7.5,NaN,91.0,17.0,94.1,0.00,2590.0,NaN,60.7,94.8,NaN,80.8,NaN,79.5,92.1,100.0,1.0,NaN
2,32229.0,5.00,7.6,NaN,528.0,186.0,69.9,3.4,1666,7.7,NaN,220,72.7,4.5,1594,7.7,NaN,272.0,92.0,78.3,0.2,1742,7.6,NaN,128.0,21.0,81.0,0.05,1888.0,NaN,58.2,95.6,NaN,52.9,NaN,75.8,88.7,98.5,1.0,NaN
3,35023.0,3.50,7.9,205.0,588.0,192.0,65.6,4.5,2430,7.8,236.0,268,73.1,8.5,2280,7.8,158.0,376.0,96.0,77.1,0.4,2060,7.6,20.0,104.0,20.0,96.7,0.00,1840.0,33.1,64.2,95.3,87.3,72.3,90.2,82.3,89.6,100.0,1.0,NaN
4,36924.0,1.50,8.0,242.0,496.0,176.0,64.8,4.0,2110,7.9,NaN,236,57.6,4.5,2020,7.8,NaN,372.0,88.0,68.2,0.2,2250,7.6,19.0,108.0,22.0,65.9,0.02,2120.0,NaN,62.7,95.6,NaN,71.0,92.1,78.2,87.5,99.5,1.0,NaN
5,38572.0,3.00,7.8,202.0,372.0,186.0,68.8,4.5,1644,7.8,NaN,248,66.1,8.5,1762,7.7,150.0,460.0,100.0,76.0,0.3,1768,7.5,20.0,100.0,28.0,82.1,0.00,1764.0,NaN,59.7,96.5,86.7,78.3,90.1,73.1,84.9,100.0,1.0,NaN
6,41115.0,6.00,7.8,NaN,552.0,262.0,64.1,5.0,1603,7.8,NaN,320,67.5,6.5,1608,7.8,192.0,376.0,122.0,72.1,0.4,1668,7.5,21.0,76.0,26.0,84.6,0.05,1703.0,NaN,61.9,93.8,89.1,79.8,NaN,86.2,90.1,99.0,1.0,NaN
7,36107.0,5.00,7.7,215.0,489.0,334.0,40.7,6.0,1613,7.6,NaN,304,53.9,8.0,1557,7.6,181.0,350.0,90.0,71.1,0.4,1596,7.5,17.0,162.0,18.0,66.7,0.00,1606.0,NaN,70.4,95.6,90.6,53.7,92.1,66.9,94.6,100.0,1.0,NaN
8,29156.0,2.50,7.7,206.0,451.0,194.0,69.1,4.5,1249,7.7,206.0,220,61.8,4.0,1219,7.7,111.0,282.0,124.0,77.4,0.3,1233,7.5,16.0,118.0,19.0,84.2,0.03,1338.0,46.1,43.6,92.5,85.6,58.2,92.2,73.8,90.2,99.4,1.0,NaN
9,39246.0,2.00,7.8,172.0,506.0,200.0,69.0,5.0,1865,7.8,208.0,248,66.1,6.5,1929,7.8,164.0,463.0,100.0,78.0,0.6,1825,7.6,19.0,157.0,27.0,87.0,0.02,1616.0,21.2,59.7,90.8,88.4,66.1,89.0,69.0,86.5,99.6,1.0,NaN


Делаем замену ? на NaN

In [17]:
data.replace('?', np.nan,inplace = True)

Затем делаем замену на медианные значения. Колонки с пропуском  более 60% нет, колонки не удаляем.

In [ ]:
data.fillna(data.median(axis=1), inplace=True)
data.info()

Считаем корреляцию

In [11]:
data.corr(method='pearson')[data.corr(method='pearson').abs() > 0.95]

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,0.972629,NaN,NaN,NaN,NaN,NaN,NaN,0.988076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,0.972786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Сильно коррелирующие признаки удаляем

In [12]:
data.drop([21, 14, 9], axis=1, inplace = True)

In [19]:
data.corr(method='pearson')[data.corr(method='pearson') > 0.95]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36
0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Делаем нормализацию значений

In [21]:
data = pd.DataFrame(StandardScaler().fit_transform(data.values))

Выводим индекс Рэнда и V-меру для различного количества кластеров. Смотрим где они близки.

In [37]:
data.index = init_data[0]
init_data.index = init_data[0]

x = data
y = init_data[39]

x_training_data, x_test_data, y_training_data, y_test_data = train_test_split(x, y, test_size = 0.3)

print('Количество кластеров\t\tПоказатель Рэнда\t\tV-мера')
for n in range(2, 40):
    kmeans = KMeans(n_clusters=n, random_state=0).fit(x_training_data)
    print('\t{:d}\t\t\t\t{:.3f}\t\t\t{:.3f}'.format(n, 
           adjusted_rand_score(y_training_data, kmeans.labels_),
           v_measure_score(y_training_data, kmeans.labels_)))

Количество кластеров		Показатель Рэнда		V-мера
	2				0.107			0.195
	3				0.103			0.245
	4				0.213			0.265
	5				0.395			0.390
	6				0.500			0.477
	7				0.534			0.495
	8				0.570			0.523
	9				0.426			0.491
	10				0.283			0.437
	11				0.266			0.437
	12				0.224			0.404
	13				0.176			0.387
	14				0.187			0.375
	15				0.160			0.381
	16				0.146			0.379
	17				0.183			0.398
	18				0.156			0.393
	19				0.156			0.393
	20				0.142			0.390
	21				0.156			0.399
	22				0.145			0.392
	23				0.124			0.375
	24				0.124			0.376
	25				0.152			0.407
	26				0.132			0.401
	27				0.103			0.392
	28				0.133			0.406
	29				0.133			0.406
	30				0.118			0.404
	31				0.117			0.400
	32				0.099			0.386
	33				0.097			0.389
	34				0.097			0.387
	35				0.097			0.389
	36				0.097			0.389
	37				0.097			0.387
	38				0.097			0.389
	39				0.089			0.381


Полученные значения показателя Рэнда и V-меры сильно отличаются. Результат не совпадает с указанным в water-treatment.names. K-means плохо справился.

Рассмотрим DBSCAN. Что получится при различных значениях параметра радиуса и минимального числа точек, которые должны образовывать плотную область.

In [35]:
print('Радиус\tМин. кол-во точек\tПоказатель Рэнда\tV-мера')
for e in [0.1, 0.2, 0.3, 0.6, 0.9, 1.2, 1.5, 3, 4, 5]:
    for m in range(2, 20):
        dbscan = DBSCAN(eps=e, min_samples=m).fit(x_training_data)
        print('{:.3f}\t\t{:d}\t\t\t{:.3f}\t\t{:.3f}'.format(e, m, 
            adjusted_rand_score(y_training_data, dbscan.labels_),
            v_measure_score(y_training_data, dbscan.labels_)))

Радиус	Мин. кол-во точек	Показатель Рэнда	V-мера
0.100		2			0.000		-0.000
0.100		3			0.000		-0.000
0.100		4			0.000		-0.000
0.100		5			0.000		-0.000
0.100		6			0.000		-0.000
0.100		7			0.000		-0.000
0.100		8			0.000		-0.000
0.100		9			0.000		-0.000
0.100		10			0.000		-0.000
0.100		11			0.000		-0.000
0.100		12			0.000		-0.000
0.100		13			0.000		-0.000
0.100		14			0.000		-0.000
0.100		15			0.000		-0.000
0.100		16			0.000		-0.000
0.100		17			0.000		-0.000
0.100		18			0.000		-0.000
0.100		19			0.000		-0.000
0.200		2			0.000		-0.000
0.200		3			0.000		-0.000
0.200		4			0.000		-0.000
0.200		5			0.000		-0.000
0.200		6			0.000		-0.000
0.200		7			0.000		-0.000
0.200		8			0.000		-0.000
0.200		9			0.000		-0.000
0.200		10			0.000		-0.000
0.200		11			0.000		-0.000
0.200		12			0.000		-0.000
0.200		13			0.000		-0.000
0.200		14			0.000		-0.000
0.200		15			0.000		-0.000
0.200		16			0.000		-0.000
0.200		17			0.000		-0.000
0.200		18			0.000		-0.000
0.200		19			0.000		-0.000
0.300		2			0.000		-0.000
0.300	

DBSCAN так же не показал приемлемого разбиения